In [179]:
import json
import os
import requests
import cv2
import numpy as np

# Шлях до JSON-файлу з даними профілю
json_file_path = 'instagram_profile_data.json'

# Папка для збереження зображень
save_folder = os.path.join('data', 'img')
os.makedirs(save_folder, exist_ok=True)

# Функція для збереження зображення
def save_image_from_url(image_url, save_path):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
            image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
            cv2.imwrite(save_path, image)
            print(f"Зображення збережено за адресою: {save_path}")
        else:
            print(f"Помилка отримання даних з URL: {image_url}, Статус: {response.status_code}")
    except Exception as e:
        print(f"Помилка під час збереження зображення з URL: {image_url}. Помилка: {str(e)}")

# Читаємо JSON-файл
with open(json_file_path, 'r', encoding='utf-8') as f:
    profile_data = json.load(f)

# Парсимо дані з JSON та зберігаємо зображення
def parse_and_save_images(data, folder):
    name = None
    for key, value in data.items():
        if key == 'id':
            name = value
        elif key == 'displayUrl' and name:
            image_url = value
            image_save_path = os.path.join(folder, f"{name}.jpg")
            save_image_from_url(image_url, image_save_path)
        elif key == 'id':
            name = value

# Запускаємо парсинг JSON-файлу та зберігаємо всі зображення
if isinstance(profile_data, list):
    for post in profile_data:
        parse_and_save_images(post, save_folder)
else:
    parse_and_save_images(profile_data, save_folder)


Зображення збережено за адресою: data\img\3446365341935558387.jpg
Зображення збережено за адресою: data\img\3454289618731199739.jpg
Зображення збережено за адресою: data\img\3450571347263631207.jpg
Зображення збережено за адресою: data\img\3461604457837108988.jpg
Зображення збережено за адресою: data\img\3456378128267119756.jpg
Зображення збережено за адресою: data\img\3465779991429220194.jpg
Зображення збережено за адресою: data\img\3451270401290414447.jpg
Зображення збережено за адресою: data\img\3459307891579321185.jpg
Зображення збережено за адресою: data\img\3466539552587099371.jpg
Зображення збережено за адресою: data\img\3447786414249922743.jpg
Зображення збережено за адресою: data\img\3445757204612097420.jpg
Зображення збережено за адресою: data\img\3441334969098868458.jpg
Зображення збережено за адресою: data\img\3438108151426360516.jpg
Зображення збережено за адресою: data\img\3440291375091146568.jpg
Зображення збережено за адресою: data\img\3441987240187352759.jpg
Зображення

In [187]:
import torch
from torchvision import models, transforms
from PIL import Image
from collections import Counter
import re
import pandas as pd

# Завантажуємо попередньо натреновану модель ResNet
model = models.resnet50(pretrained=True)
model.eval()

# Завантажуємо класові мітки ImageNet
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
labels = requests.get(LABELS_URL).json()

# Перетворення для передобробки зображень
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Читаємо JSON-файл
json_file_path = 'instagram_profile_data.json'
with open(json_file_path, 'r', encoding='utf-8') as f:
    profile_data = json.load(f)

# Список слів, які ми ігноруємо
stop_words = {
    "the", "a", "an", "and", "or", "but", "if", "while", "although", "because", "when", "where", "why", 
    "how", "as", "in", "on", "at", "by", "with", "to", "from", "of", "for", "about", "into", "through", 
    "during", "before", "after", "over", "under", "between", "among", "across", "against", "along", 
    "throughout", "near", "around", "toward", "towards", "since", "until", "up", "down", "out", "off", 
    "above", "below", "upon", "than", "so", "such", "that", "these", "those", "this", "some", "any", 
    "all", "few", "many", "much", "none", "most", "several", "own", "which", "what", "whose", 
    "who", "whom", "where", "why", "how", "can", "could", "would", "should", "might", "must", 
    "may", "will", "shall", "do", "does", "did", "has", "have", "had", "is", "are", "was", "were", 
    "be", "been", "being", "seems", "seemed", "seeming", "does", "do", "done", "go", "goes", 
    "went", "get", "got", "having", "each", "every", "either", "neither", "not", "no", "nor", "yes", 
    "both", "more", "less", "even", "very", "here", "there", "always", "often", "sometimes", 
    "never", "then", "now", "just", "only", "ever", "yet", "still", "already", "too", "also", 
    "perhaps", "maybe", "probably", "quite", "actually", "often", "rather", "further", "either", 
    "neither", "again", "another", "whenever", "whoever", "whatever", "wherever", "whenever", 
    "therefore", "however", "besides", "furthermore", "meanwhile", "likewise",
    "i", "you", "me", "he", "she", "it", "we", "they", "us", "him", "her", "them", "my", "your", 
    "his", "their", "our", "mine", "yours", "hers", "theirs", "ours", "myself", "yourself", 
    "himself", "herself", "itself", "ourselves", "yourselves", "themselves", "thank", "know"
}

# Функція для отримання підпису (caption) за id
def get_caption_by_id(image_id, data):
    for post in data:
        if post.get('id') == image_id:
            return post.get('caption', '')
    return ''

# Функція для передбачення об'єктів на зображенні
def predict_image(image_path):
    img = Image.open(image_path)
    img_tensor = preprocess(img).unsqueeze(0)

    with torch.no_grad():
        output = model(img_tensor)
    
    # Отримуємо ідентифікатор класу
    _, predicted = torch.max(output, 1)
    return labels[predicted.item()]

# Функція для підрахунку кількості слів у підписі з виключенням
def count_words_in_caption(caption):
    words = re.findall(r'\w+', caption.lower())  # розділяємо на слова і приводимо до нижнього регістру
    filtered_words = [word for word in words if word not in stop_words]  # Ігноруємо стоп-слова
    return Counter(filtered_words)

# Функція для аналізу всіх зображень у папці
def analyze_images(image_folder, data):
    object_counts = Counter()
    word_counts = Counter()
    
    for image_file in os.listdir(image_folder):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_id = image_file.split('.')[0].split('_')[0]  # Отримуємо ID з імені файлу до ".jpg" або "_"
            image_path = os.path.join(image_folder, image_file)

            # Розпізнаємо об'єкт на зображенні
            predicted_class = predict_image(image_path)
            object_counts[predicted_class] += 1

            # Отримуємо підпис за ID і рахуємо слова
            caption = get_caption_by_id(image_id, data)
            if caption:
                word_counts.update(count_words_in_caption(caption))

    return object_counts, word_counts

# Шлях до папки з зображеннями
image_folder = 'data/img'

# Аналізуємо всі зображення і рахуємо кількість об'єктів та слів у підписах
object_counts, word_counts = analyze_images(image_folder, profile_data)

# Збереження статистики об'єктів та слів у змінні
sorted_object_counts = object_counts.most_common()

# Залишаємо лише ті слова, які зустрічаються більше 3 разів
filtered_word_counts = [(word, count) for word, count in word_counts.most_common() if count > 3]

# Створення таблиць об'єктів та слів
object_df = pd.DataFrame(sorted_object_counts, columns=['Object', 'Count'])
word_df = pd.DataFrame(filtered_word_counts, columns=['Word', 'Count'])

# Виводимо результат за об'єктами у вигляді таблиці (перші 5 рядків)
print("\nТоп 5 об'єктів на зображеннях (відсортовано за спаданням):")
print(object_df.head(5))

# Виводимо результат аналізу підписів у вигляді таблиці (перші 5 рядків)
if not word_df.empty:
    print("\nТоп 5 слів у підписах :")
    print(word_df.head(5))
else:
    print("\nНе знайдено слів для аналізу.")

# Знаходимо об'єкт, якого найбільше
if sorted_object_counts:
    most_common_object = sorted_object_counts[0]
    print(f"\nНайчастіший об'єкт: {most_common_object[0]} з кількістю {most_common_object[1]}")

# Знаходимо найчастіше слово в підписах
if filtered_word_counts:
    most_common_word = filtered_word_counts[0]
    print(f"\nНайчастіше слово в підписах: '{most_common_word[0]}' з кількістю {most_common_word[1]}")


c:\Users\Dmitry\Documents\Repos\TestTaskInstaQuestion\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Dmitry\Documents\Repos\TestTaskInstaQuestion\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Топ 5 об'єктів на зображеннях (відсортовано за спаданням):
      Object  Count
0        bra      7
1       gown      5
2  miniskirt      4
3     tights      4
4  tank suit      3

Топ 5 слів у підписах :
    Word  Count
0  skims     12
1   best      8
2   life      5
3      m      5
4   love      4

Найчастіший об'єкт: bra з кількістю 7

Найчастіше слово в підписах: 'skims' з кількістю 12


In [191]:
from dotenv import load_dotenv
from openai import OpenAI

# Завантажуємо змінні середовища з файлу .env
load_dotenv()

# Перевіряємо, чи завантажився API-ключ
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"API ключ завантажено: {api_key[:5]}...")  # Виводимо перші 5 символів для перевірки
else:
    print("API ключ не знайдено. Перевірте файл .env")

# Ініціалізуємо клієнт OpenAI
client = OpenAI(api_key=api_key)

# Отримуємо три найбільш поширених об'єкти
top_3_objects = object_df.head(3)['Object'].tolist()

# Отримуємо три найбільш поширених слова
top_3_words = word_df.head(3)['Word'].tolist()

# Поєднуємо об'єкти та слова
combined_top_words = top_3_objects + top_3_words

# Формуємо повідомлення з цими словами
message_content = f"What do the following words have in common: {', '.join(combined_top_words)}."

# Відправляємо запит до GPT-4 для генерації речення з цими словами
stream = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": message_content}],
    stream=True,
)

# Читаємо відповідь у потоковому режимі та зберігаємо результат
message_content1 = ""  # Ініціалізуємо змінну для збереження відповіді
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        message_content1 += chunk.choices[0].delta.content  # Збираємо весь текст відповіді

# Тепер генеруємо відповідне повідомлення для знайомства, використовуючи message_content1
final_message_content = (
    f"Generate a suitable introduction message for getting to know this person about {message_content1} and {combined_top_words} "
    "You don't have to use all the words. Do not use swear words. "
    "Important, The sentence should between 25 and 50 characters, no more."
)

# Відправляємо другий запит до GPT-4 для створення повідомлення для знайомства
stream2 = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": final_message_content}],
    stream=True,
)

# Читаємо другий потік і виводимо результат
final_message = ""  # Ініціалізуємо змінну для збереження другої відповіді
for chunk in stream2:
    if chunk.choices[0].delta.content is not None:
        final_message += chunk.choices[0].delta.content  # Збираємо весь текст відповіді

# Виводимо фінальне згенероване повідомлення для знайомства
print(final_message)


API ключ завантажено: sk-pr...
"Skims lover who lives the Good American life."
